In [1]:
# !pip install pytorch
# !pip install transformers
#! sudo apt install tesseract-ocr
#! pip install pytesseract
#! pip install transformers datasets

In [2]:
# !pip install nltk

In [1]:
from transformers import BertModel
from transformers import BertTokenizer
import torch


from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix,ConfusionMatrixDisplay
from sklearn.preprocessing import LabelEncoder

import os
import numpy as np
# tqdm.tqdm(dirs, desc='dirs') 
from tqdm import tqdm
import pandas as pd
#import itertools
from PIL import Image
import pytesseract
import re
import nltk
#nltk.download('wordnet')
#nltk.download('stopwords')
#nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer #tokanization and removel of punctuation 

import warnings
warnings.filterwarnings('ignore')

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#classes
txt_path='/media/umar_visionx/Backup Plus/Active/Faizan/doc_classifier_classes.txt'
with open(txt_path,'r') as file:
    classes=file.read().split('\n')
print(classes)


['Admin Note', 'Clinical History And Summary', 'Consult Note', 'Cover Page', 'Imaging Note', 'Insurance Authorization', 'Intake Forms', 'Lab Test', 'Other', 'Patient Profile', 'Prescriptions', 'Referral Letter', 'Requisition Form']


In [3]:
# #to extract text from the document we are applying OCR to each document
# def apply_ocr(sample):

#         # apply ocr to the image
#         #Extracting words and bounding boxes from images
#         ocr_dframe = pytesseract.image_to_data(sample, output_type='data.frame')
#         #return a subset of the DataFrame's columns based on the column dtypes.
#         float_cols = ocr_dframe.select_dtypes('float').columns
#         #dropping the rows where atleast one element is not present(i.e "text" is not present)
#         ocr_dframe = ocr_dframe.dropna().reset_index(drop=True)
#         #selecting float_cols,round along zero,changing type to integer
#         ocr_dframe[float_cols] = ocr_dframe[float_cols].round(0).astype(int)
#         #replace field that's entirely space (or empty) with NaN check https://stackoverflow.com/questions/13445241/replacing-blank-values-white-space-with-nan-in-pandas
#         ocr_dframe = ocr_dframe.replace(r'^\s*$', np.nan, regex=True)
#         #dropping the rows where atleast one element is not present(i.e "text" is not present)
#         ocr_dframe = ocr_dframe.dropna().reset_index(drop=True)

#         # get the words and actual (unnormalized) bounding boxes
#         #words = [word for word in ocr_dframe.text if str(word) != 'nan'])
#         words = list(ocr_dframe.text)#making the list of column "text" in 'ocr_dframe' dataframe
#         #words = [str(w) for w in words] #converting into list of string of words from the last words list 

#         #making a list of words in the image
#         image_words=[]
#         for word in words:
#           if str(word)!="nan":
#             image_words.append(str(word))

      
#         #joining words with space
#         words= " ".join(image_words)
#         return words

In [4]:
# #making folders and saving the text files (with extracted text from image) in related folders
# def extract_text(source_path,dest_path):
#     #first making folders
#     try:
#         os.makedirs(dest_path)
#         for ent in classes:
#             folder_path=os.path.join(dest_path,ent)
#             os.makedirs(folder_path)
#             print("Folder making process completed")
#     except:
#         pass
#     for folder in os.listdir(source_path):
#         folder_path=os.path.join(source_path,folder)
#         for files in os.listdir(folder_path):
#             f_path=os.path.join(folder_path,files)#path of image
#             image = Image.open(f_path)
#             image = image.convert("RGB")
#             para=apply_ocr(image) #getting words from image
        
#         #making directory to save the files with same file and folder name
#             txt_file=os.path.splitext(files)[0] + ".txt" #making same name of file as image
#             txt_path=os.path.join(dest_path,os.path.join(folder,txt_file))
#         #writing and saving file
#             with open(txt_path, "w") as file:
#                 file.write(para)
#     print("All text files are created successfully")

In [5]:
# #extracting text from images and creating text files 
# image_data_source="/media/umar_visionx/Backup Plus/Active/Faizan/image_data"#image data source path
# test_image_source="/media/umar_visionx/Backup Plus/Active/Faizan/test_images"#test image data source path
# text_dest="/media/umar_visionx/Backup Plus/Active/Faizan/dataset"#extracted text files destinatio path for image data
# test_text_dest="/media/umar_visionx/Backup Plus/Active/Faizan/test_dataset"#extracted text files destination path from test image data

# extract_text(image_data_source,text_dest)
# extract_text(test_image_source,test_text_dest)

In [3]:
#removing special character and numbers
def rem_sp_char(words):
    pattern = r'[^a-zA-z0-9.,!?/:;\"\'\s]'  # defined pattern to keep
    words=re.sub(pattern, '', words)
    return words

#tokanizing and removing punctuations
def remove_punct(words):
  tokenizer = RegexpTokenizer(r'\w+')
  tokens=tokenizer.tokenize(words)
  return tokens

#lemmitization
def lemmitizar(words):
  lemmatized_words=[]
  l = WordNetLemmatizer()
  for word in words:
      token = l.lemmatize(word)
      lemmatized_words.append(token)
  return lemmatized_words

#removing stop-words
def remove_stopwords(words):
  filter_words = []
  Stopwords = stopwords.words('english')
  for word in words:
      if word not in Stopwords:
          filter_words.append(word)
  return filter_words

#converting to lowercase
def lower_case(words):
  lower_words=[]
  for word in words:
    lower_words.append(word.lower())
  return lower_words

In [4]:
#path of text files
data_path='/media/umar_visionx/Backup Plus/Active/Faizan/dataset'
test_path='/media/umar_visionx/Backup Plus/Active/Faizan/test_dataset'

In [5]:
#label encoding
labl=[]
for folder in os.listdir(data_path):
  labl.append(folder)#making the list of folder names which are labels


#encoding the labels from the list
label_index={}
for index,label in enumerate(labl):
  label_index[label]=index

label_index#label to index

{'Admin Note': 0,
 'Clinical History And Summary': 1,
 'Consult Note': 2,
 'Cover Page': 3,
 'Imaging Note': 4,
 'Insurance Authorization': 5,
 'Intake Forms': 6,
 'Lab Test': 7,
 'Other': 8,
 'Patient Profile': 9,
 'Prescriptions': 10,
 'Referral Letter': 11,
 'Requisition Form': 12}

In [6]:
#making dictionary for label decoding
index_label={}
for key,val in zip(label_index.keys(),label_index.values()):
    index_label[val]=key
print(index_label)

{0: 'Admin Note', 1: 'Clinical History And Summary', 2: 'Consult Note', 3: 'Cover Page', 4: 'Imaging Note', 5: 'Insurance Authorization', 6: 'Intake Forms', 7: 'Lab Test', 8: 'Other', 9: 'Patient Profile', 10: 'Prescriptions', 11: 'Referral Letter', 12: 'Requisition Form'}


In [7]:
#making dictionaries and lists of text and labels
def preprocessed_df(data_path):
    data_dict={}
    #txt_file_path=[]
    labels=[]
    text=[]
    encoded_labels=[]
    for clss in os.listdir(data_path):
        clss_path=os.path.join(data_path,clss)
        for txt_file in os.listdir(clss_path):
            file_path=os.path.join(clss_path,txt_file)
            #reading text file
            with open(file_path, "r") as file:
                para = file.read().rstrip('\n')#rstrip method removes trailing character based on string argument passed

            #apply preprocessing functions on para
                words = rem_sp_char(para) #removing special character and numbers
                tokens = remove_punct(words) #removing punctuations and tokanization
                tokens = lemmitizar(tokens) #lemmitization
                tokens = remove_stopwords(tokens) ##removing stop-words
                tokens = lower_case(tokens) #converting to lowercase

            #joining words with space and removing single tetter tokens
                parag=' '.join( [word for word in tokens if len(word)>1] )
            #parag=' '.join( [word for word in tokens if len(word)>1 and word.isalpha()] )#if we dont want numeric values


            #inseting data into lists and dictionaries
                text.append(parag)
            #text_file_path.append(file_path)
                labels.append(clss)
                data_dict[parag]=clss
#label encoding
    for ent in labels:
        encoded_labels.append(label_index[ent])
    


    df=pd.DataFrame()
    df['text']=text
    df['labels']=encoded_labels
    return df

In [8]:
df=preprocessed_df(data_path)
df_test=preprocessed_df(test_path)
df.head()

,text,labels
0,29 2021 10 42 alberta health services rrduser ...,0
1,30 2021 09 34 alberta health services rrduser ...,0
2,29 2021 11 03 alberta health services rrduser ...,0
3,2021 08 25 alberta health services rrduser 403...,0
4,ahs 2021 23 05 am page alberta health services...,0


In [9]:
print(len(df))
print(len(df_test))

11660
1810


In [10]:
#checking null values in dataframes
print(df.isnull().any().sum())
print(df_test.isnull().any().sum())

0
0


In [11]:
print(f"Unique Labels after Encoding:  {df.labels.unique()}")

Unique Labels after Encoding:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12]


In [12]:
#BERT base model
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased',output_hidden_states = True,)#model returns all hidden-states.
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(df)
#val_dataset = Dataset.from_pandas(df_val)
test_dataset = Dataset.from_pandas(df_test)

In [14]:
def encode_example(sample,max_seq_length=512):
    words=sample['text']
    label=sample['labels']
    #splitting all words of sentence in to into tokens
    tokenized_text = tokenizer.tokenize(words)
    # Truncation of token_boxes
    special_tokens_count = 2 
    if len(tokenized_text) > max_seq_length - special_tokens_count:
      tokenized_text = tokenized_text[: (max_seq_length - special_tokens_count)]#to make place available for cls and sep tokens
    # Adding CLS and SEP tokens
    input_sequence = ["[CLS]"] + tokenized_text + ["[SEP]"]
    # Map the tokens to their vocabulary indeces
    indexed_tokens = tokenizer.convert_tokens_to_ids(input_sequence)
    #padding length
    padding_length = max_seq_length - len(input_sequence)
    indexed_tokens += [0] * padding_length#zero padding to maximum length
    #as we are giving as a single sentence so mark each token as belonging to sentence 1
    segments_ids = [1] * max_seq_length
    pad_masks = [1] * len(input_sequence) + [0] * padding_length


    assert len(indexed_tokens) == max_seq_length
    assert len(segments_ids) ==max_seq_length
    assert len(pad_masks) == max_seq_length

    #converting into pytorch tensors
    tokens_tensor = torch.tensor(indexed_tokens)
    segments_tensors = torch.tensor(segments_ids)
    pad_mask_tensors=torch.tensor(pad_masks)
    label_tensors=torch.tensor(label)
    
  
    tok_tensors.append(tokens_tensor)
    seg_tensors.append(segments_tensors)
    lab_tensors.append(label_tensors)
    pad_msk_tensors.append(pad_mask_tensors)


In [16]:
def iter_df(dataset):
    global tok_tensors,seg_tensors,lab_tensors,pad_msk_tensors

    tok_tensors=[]
    seg_tensors=[]
    lab_tensors=[]
    pad_msk_tensors=[]
    dataset.map(encode_example)
    # df=pd.DataFrame()
    # df['token_tensors']=tok_tensors
    # df['segment_tensor']=seg_tensors
    # df['labels']=lab_tensors
    return tok_tensors,seg_tensors,lab_tensors,pad_msk_tensors

In [17]:
tr_token_tensor,tr_segment_tensors,tr_label_tensors,tr_pad_msk_tensors=iter_df(train_dataset)
ts_token_tensor,ts_segment_tensors,ts_label_tensorsts,ts_pad_msk_tensors=iter_df(test_dataset)

  0%|          | 0/11660 [00:00<?, ?ex/s]

  0%|          | 0/1810 [00:00<?, ?ex/s]

In [18]:
from torch.utils.data import DataLoader,Dataset
class PhelixDataset(Dataset):
    def __init__(self, token_tens,label_tens,seg_tens,pd_msk_tens):
        #super().__init__()
        self.token_tens = token_tens
        self.seg_tens=seg_tens
        self.label_tens=label_tens
        self.pd_msk_tens=pd_msk_tens

    def __len__(self):
        return len(self.token_tens)
    
    def __getitem__(self, index):       
        tokens_id = self.token_tens[index]
        segment_id = self.seg_tens[index]
        label=self.label_tens[index]
        pad_mask=self.pd_msk_tens[index]

        return tokens_id,label,segment_id,pad_mask

In [19]:
# from torch.utils.data import DataLoader,Dataset
# class PhelixDataset(Dataset):
#     def __init__(self, token_tens,seg_tens):
#         #super().__init__()
#         self.token_tens = token_tens
#         self.seg_tens=seg_tens

#     def __len__(self):
#         return len(self.token_tens)
    
#     def __getitem__(self, index):       
#         tokens_id = self.token_tens[index]
#         segment_id = self.seg_tens[index]

#         return tokens_id,segment_id

In [20]:
batch_size=1
train_data = PhelixDataset(tr_token_tensor,tr_label_tensors,tr_segment_tensors,tr_pad_msk_tensors)
test_data = PhelixDataset(ts_token_tensor,ts_label_tensorsts,ts_segment_tensors,ts_pad_msk_tensors)
train_dataloader = DataLoader(train_data, batch_size = batch_size,shuffle=True,num_workers=3,pin_memory=True)
test_dataloader = DataLoader(test_data, batch_size = batch_size,shuffle=False,num_workers=3,pin_memory=True)
sample = next(iter(train_dataloader))
print(sample)
# batch=next(iter(train_dataloader))
# print(batch)

[tensor([[  101,  2340,  2423, 12609,  2403,  2322, 12458,  2094,  9634,  6904,
          2595,  6904,  2015,  6564, 28756, 24594, 21486, 24434,  5776,  2592,
         19739, 20486,  4263,  2592,  2171,  2879,  3401,  2798,  2171,  3348,
          2128,  2140,  2969,  7020,  2078,  7020,  2078,  2720,  2078,  8732,
          2620, 18827,  2575,  4769,  6146, 17788,  4496,  5243,  3367,  2852,
          4769,  4769,  4769,  2103,  2358,  4138,  3122,  4564, 19067,  2103,
          2358, 14101,  6146, 15136,  2692,  2575, 17914,  2509, 14101,  7026,
          6282,  2581,  2575,  2683, 18827, 24096,  2575,  7026,  3058,  4182,
          5641,  2403,  3874,  5427,  2592,  1042, 11020,  2171,  8292,  5339,
          2053,  2177,  2933,  4942, 29234,  2099,  2128,  2140, 18253,  1051,
         27487, 27615, 24528,  2278,  1021,  2546,  2290,  2620, 10374,  2546,
          2078, 18827,  2879,  3401,  2798,  7367, 27908,  9779, 14536, 27615,
         10514,  9397,  2571, 23980, 26187, 16703, 

In [21]:
for token, label,seg,pad_mask in train_dataloader:
    print(token.shape)
    print(label.shape)
    print(seg.shape)
    print(pad_mask.shape)
    break

torch.Size([1, 512])
torch.Size([1])
torch.Size([1, 512])
torch.Size([1, 512])


In [22]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')
model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [23]:
import torch
torch.cuda.empty_cache()

In [24]:
#testing GPU
!nvidia-smi

Mon Mar  7 14:51:13 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:65:00.0  On |                  N/A |
|  0%   35C    P2    68W / 320W |   3693MiB / 11264MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [32]:
def embed(data_loader):
  #creating word embeddings
  model.eval()
  # token_labels_list=[]
  # concat_token_vect=[]#define list to append concatinated token vectors
  # doc_vectors_list=[]
  # doc_label_list=[]
  #to save data as arrays 
  token_labels_list_arr=[]
  concat_token_vect_arr=[]
  doc_vectors_list_arr=[]
  doc_label_list_arr=[]




  for tokens,label,seg_id,_ in tqdm(data_loader):
  
      #moving to device
      label=int(label)
      tokens=tokens.to(device)
      seg_id=seg_id.to(device)
      with torch.no_grad():
        output = model(tokens,seg_id)
      #we set output_hidden_state=True in our pre trained model so, we will get the hidden states from all 13 layers at third positiob
      hidden_states = output[2]
      #For futher info please check https://mccormickml.com/2019/05/14/BERT-word-embeddings-tutorial/
      # Concatenating tensors for all layers and creating new dim with the help of stack
      token_embed = torch.stack(hidden_states, dim=0)#token embeddings(torch.Size([13, 1, 512, 768]))
      token_embed = torch.squeeze(token_embed, dim=1)#removing batch dimension(torch.Size([13, 512, 768]))
      token_embed = token_embed.permute(1,0,2)# swapping dimensions(torch.Size([512, 13, 768]))
      #iterating for each token in whole document/sentense
      to_append_vect=[]
      to_append_vect_arr=[]
      #to_append_label=[]
      for token in token_embed:
          #token vector of size 13*768 where 13 are number of layers while we want to get only last 4 layers i.e 4*768
          #after concatination last four layers vector dimensions will be 4*768=3,072(singlre word vector length i.e per token)
          #concatinating vectors
          add1 = torch.add(token[-1], token[-2])
          add2=torch.add(token[-3], token[-4])
          full_vector=torch.add(add1,add2)
          #concatcat_vector = torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0)
          # concat_token_vect.append(full_vector)
          #we have append 512 vectors of size 768 for each document
          # token_labels_list.append(label)
          token_labels_list_arr.append(label)
          concat_token_vect_arr.append(full_vector.cpu().numpy())

          to_append_vect_arr.append(full_vector.cpu().numpy())
          to_append_vect.append(full_vector)
          #to_append_label.append(label)
      # print(len(to_append_vect_arr))
      # print(len(to_append_vect))


      # doc_vectors_list.append(to_append_vect)
      # doc_label_list.append(label)
      doc_vectors_list_arr.append(to_append_vect_arr)
      doc_label_list_arr.append(label)
  return doc_vectors_list_arr,doc_label_list_arr

In [37]:
train_doc_vectors_list,train_doc_label_list = embed(train_dataloader)
test_doc_vectors_list,test_doc_label_list = embed(test_dataloader)

In [38]:
#this list have document vectors,every vector,every document have 512 token vectors and every oken vector have 3072 entries
print(len(train_doc_vectors_list[0:10]))
print(len(train_doc_label_list))
print(len(train_doc_vectors_list[1]))
print(len(train_doc_label_list))
print(len(train_doc_vectors_list[1][1]))